In [78]:
import os

In [79]:
%pwd

'/Users'

In [80]:
os.chdir("../")

In [85]:
%pwd

'/'

In [86]:
#/Users/gowthambujju/Documents/Gowtham_pro/AI_projects/Text-summarizer-project/.github
os.chdir("/Users/gowthambujju/Documents/Gowtham_pro/AI_projects/Text-summarizer-project")

In [87]:
%pwd

'/Users/gowthambujju/Documents/Gowtham_pro/AI_projects/Text-summarizer-project'

In [88]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [82]:
from src.textsummarizer.constants import *
from src.textsummarizer.utils.common import read_yaml, create_directories

In [89]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [90]:
ConfigurationManager()

[2023-12-26 19:22:20,659 : INFO:common:yaml file: config/config.yaml loaded successfully]
[2023-12-26 19:22:20,660 : INFO:common:yaml file: params.yaml loaded successfully]
[2023-12-26 19:22:20,661 : INFO:common:created directory at: artifacts]


In [ ]:
import os
import urllib.request as request
import zipfile
from src.textsummarizer.logging import logger
from src.textsummarizer.utils.common import get_size

In [92]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [94]:
#Creating Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-26 19:24:19,752 : INFO:common:yaml file: config/config.yaml loaded successfully]
[2023-12-26 19:24:19,754 : INFO:common:yaml file: params.yaml loaded successfully]
[2023-12-26 19:24:19,754 : INFO:common:created directory at: artifacts]
[2023-12-26 19:24:19,755 : INFO:common:created directory at: artifacts/data_ingestion]
[2023-12-26 19:24:22,041 : INFO:2523701424:artifacts/data_ingestion/data.zip download! with following info: 
x-amz-id-2: Q8r0P4YxATqoGAEiTd1K8XW0jg2WId/Mt2j2RgTxzvwVOC8dRd16BxCI0rCdvdIfXKEBIxAwPok=
x-amz-request-id: 1SS68EFHY0VJR38A
Date: Tue, 26 Dec 2023 13:54:21 GMT
Last-Modified: Tue, 26 Dec 2023 13:22:15 GMT
ETag: "8e8de8d8666735de004076151c68cd8d"
x-amz-server-side-encryption: AES256
Accept-Ranges: bytes
Content-Type: application/zip
Server: AmazonS3
Content-Length: 7903594
Connection: close

]
